In [1]:
# Importing the libraries 
import pandas as pd

In [2]:
rootpath = "/hdd/work/d4ad_standardization/"
filepath = "./D4AD_Standardization/data/raw/etpl_all_programsJune3.xls"

columns = [
    "NAME",
    "NAME_1",
    "DESCRIPTION",
    "PREREQUISITES",
    "FEATURESDESCRIPTION",
    "STREET1",
    "CITY",
    "STATE",
    "ZIP",
    "WEBSITE",
    "COUNTY",
    "NONGOVAPPROVAL",
    "STATECOMMENTS",
    "CIPCODE",
    "PROVIDERID",
    "APPROVINGAGENCYID"
]

df = pd.read_excel(rootpath + filepath, usecols=columns)
print('done')

done


In [81]:
processedpath = "./D4AD_Standardization/data/processed/"

content_is = "NAME_lookup.csv"

the_df = df # df.sample(n=100, random_state=42)

columns_to_save = ['STANDARDIZED_NAME', 'NAME', 'PROVIDERID',
           'APPROVINGAGENCYID', 'CIPCODE']

"""
# regex from start of string up to and excluding first dash character
regex_pattern = r'\A([^-]+)\s?'

the_df['STANDARDIZED_NAME'] =\
    the_df.NAME.str.extract(pat=regex_pattern)
the_df['STANDARDIZED_NAME'] =\
    the_df['STANDARDIZED_NAME'].str.rstrip() # can't easily get that trailing space out from regex
"""

the_df['STANDARDIZED_NAME'] =\
    the_df.NAME.str.split(" - ", n=1)[0]


# So the regex r'\A([^-]+)\s? almost completely works but
# fails in cases of ABC-DEF - NJ and it is more straightforward
# to split on " - " and get take the first result.



# copy over null matches
index = the_df.STANDARDIZED_NAME.isna()
the_df[index].STANDARDIZED_NAME = the_df[index].NAME
"""
the_df.to_csv(rootpath + processedpath + "{}".format(content_is),
              index = False,
              chunksize = 10000,
              columns=columns_to_save)
              """
print('done')

done


In [96]:
the_df =  df.sample(n=100, random_state=42)
the_df.iloc[0:60].NAME.str.split(" - ", n=1)


3494              [Star Technical Institute, Edison - PVS]
14560    [New Horizons of Central and Northern NJ, Inc....
5535                   [KeySkills Learning, Inc., Maywood]
20964                                   [Beyond Today, TP]
17769            [Rutgers Center for Innovation Education]
22724               [North Jersey Massage Training Center]
4720                         [Essex County College Credit]
12570    [Rowan College at Gloucester County Division o...
4186        [Raritan Valley Community College, Non Credit]
2270              [Newark Business Training Institute, TP]
16161                                   [ACECS, Inc., PCS]
4043      [Rowan College at Burlington County, Non-credit]
23850    [Universal Technical Institute Northeast, LLC,...
23293    [The New York Code and Design Academy- Jersey ...
16700    [CDM Technical Training Institute, Inc., Phill...
17700          [1st Security Preparation & Placement, PVS]
2174             [Union County College, Academic Program

In [93]:
the_df.NAME

3494               Star Technical Institute - Edison - PVS
14560    New Horizons of Central and Northern NJ, Inc. ...
5535                    KeySkills Learning, Inc. - Maywood
20964                                    Beyond Today - TP
17769              Rutgers Center for Innovation Education
                               ...                        
14546              Rutgers Center for Innovation Education
8231     Lincoln Technical Institute - S. Plainfield - PCS
17598                          Future Media Concepts, Inc.
13871                       American College of Technology
2714     Newark Business Training Institute -Title II G...
Name: NAME, Length: 100, dtype: object

The above does not work in cases where:
* There is not match (empty)
* When there is no dash, ([^-])+ greedily matches up to the last space, often dropping the end of the name